In [3]:
import pathlib

import SessionTools as st
import SessionTools.two_photon as st2p

%load_ext autoreload
%autoreload 2 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
date = "22_03_2023"
fly = "EPG_GCaMP_EL_CsChrimsontdT_fly2"
session = "dangling_alternating_stim_group1-004"
basedir = pathlib.PurePath(f'/media/mplitt/SSD_storage/2PData/{date}/{fly}/{session}/')
basename_input = pathlib.Path(basedir.joinpath(session))
metadata = st2p.preprocessing.bruker_metadata.read(basename_input)

In [28]:
import os
import logging
import pathlib

import numpy as np
import dask
from dask import diagnostics
import dask.array as da

from skimage.io import imread
from SessionTools import utilities

logger = logging.getLogger(__name__)

class TiffToolsError(Exception):
    """Error while processing tiff files"""

def read(base, size, layout):
    """Read 2p dataset of TIFF files into a dask.Array."""
    # shape_yx = (size['y_px'], size['x_px'])
    # dtype = read_file(base, 0, channel, 0).dtype

    num_cycles, num_z_planes, num_ch = layout['sequences'], size['z_planes'], size['channels']
    
    def skimread(file):
        """suppress juliandate reading errors"""
        with utilities.suppress_output(suppress_stderr=True):
            return imread(file)
    
    frame, cycle, ch = 1, 1, 1
    sample = skimread(str(base) + f'_Cycle{cycle:05d}_Ch{ch}_{frame:06d}.ome.tif')
    if len(sample.shape)==2:
        ome_tiff=False
    elif len(sample.shape)==3:
        ome_tiff=True
    else:
        raise "tiff of unknown shape"
        
                      
    filenames = []
    if not ome_tiff:
        for cycle in range(1,num_cycles+1):
            _filenames = []
            for frame in range(1,num_z_planes+1):
                _frame = []
                for ch in range(1,num_ch+1):
                    _frame.append(str(base) + f'_Cycle{cycle:05d}_Ch{ch}_{frame:06d}.ome.tif')
                _filenames.append(_frame)
            # filenames[cycle].append(str(basename_input) + f'_Cycle{cycle+1:05d}_Ch{channel}_{frame+1:06d}.ome.tif')
            filenames.append(_filenames)    
    else:
        frame=1
        for cycle in range(1,num_cycles+1):
            _filenames = []
            for ch in range(1,num_ch+1):
                _frame = str(base) + f'_Cycle{cycle:05d}_Ch{ch}_{frame:06d}.ome.tif'
                _filenames.append(_frame)
            # filenames[cycle].append(str(basename_input) + f'_Cycle{cycle+1:05d}_Ch{channel}_{frame+1:06d}.ome.tif')
            filenames.append(_filenames)   
        
    
    logger.info('Found tiff files (channels: %i, frames: %i, z_planes: %i' % (num_cycles, num_z_planes, num_ch))
    
    def read_one_image(block_id, filenames=filenames):
        # print(block_id)
        path = filenames[block_id[1]][block_id[2]][block_id[0]]
        image = skimread(path)
        return np.expand_dims(image, axis=(0,1,2))
    
    def read_one_image_ome(block_id, filenames=filenames):
        # print(block_id)
        path = filenames[block_id[1]][block_id[0]]
        image = skimread(path)
        return np.expand_dims(image, axis=(0,1))
    
    logger.info('Mapping dask array...')
    if not ome_tiff:
        sample = skimread(filenames[0][0][0])
        data = da.map_blocks(
            read_one_image,
            dtype=sample.dtype,
            chunks=((1,)*num_ch,
                    (1,)*num_cycles, 
                    (1,)*num_z_planes, 
                    *sample.shape)
            )
        
    else:
        sample = skimread(filenames[0][0])
        data = da.map_blocks(
            read_one_image_ome,
            dtype=sample.dtype,
            chunks=((1,)*num_ch,
                    (1,)*num_cycles,  
                    *sample.shape)
            )
    logger.info('Completed mapping dask array')
    
    return data

In [31]:
tiff_data = read(basename_input, metadata['size'],metadata['layout'])

In [32]:
tiff_data.shape

(2, 4166, 8, 202, 234)

In [18]:
from glob import glob
print(glob(str(basename_input)+"_Cycle00001_Ch1_000001.ome.tif"))
test_tiffname = str(basename_input)+"_Cycle00001_Ch1_000001.ome.tif"
tiff = imread(test_tiffname)

['/media/mplitt/SSD_storage/2PData/22_03_2023/EPG_GCaMP_EL_CsChrimsontdT_fly2/dangling_alternating_stim_group1-004/dangling_alternating_stim_group1-004_Cycle00001_Ch1_000001.ome.tif']


In [19]:
tiff.shape

(8, 202, 234)

In [4]:
base = basename_input
layout, size = metadata['layout'], metadata['size']
num_cycles, num_z_planes, num_ch = layout['sequences'], size['z_planes'], size['channels']
    
def skimread(file):
    """suppress juliandate reading errors"""
    with utilities.suppress_output(suppress_stderr=True):
        return imread(file)
    
filenames = []
for cycle in range(2,num_cycles+1):
    _filenames = []
    for frame in range(1,num_z_planes+1):
        _frame = []
        for ch in range(1,num_ch+1):
            _frame.append(str(base) + f'_Cycle{cycle:05d}_Ch{ch}_{frame:06d}.ome.tif')
        _filenames.append(_frame)
    # filenames[cycle].append(str(basename_input) + f'_Cycle{cycle+1:05d}_Ch{channel}_{frame+1:06d}.ome.tif')
    filenames.append(_filenames)    

NameError: name 'metadata' is not defined

In [47]:
def read_one_image(block_id, filenames=filenames):
        # print(block_id)
        path = filenames[block_id[1]][block_id[2]][block_id[0]]
        image = skimread(path)
        return np.expand_dims(image, axis=(0,1,2))

In [51]:
filenames[0][0][0]

'/media/mplitt/SSD_storage/2PData/12_16_2022/EPG_GCaMP7f_CsChrimsontdT_fly1/fixed_cw_remapping-000/fixed_cw_remapping-000_Cycle00002_Ch1_000001.ome.tif'

In [49]:
data = imread(filenames[1][0][0])

In [50]:
data.shape

(208, 259)

In [29]:
sample = skimread(filenames[0][0][0])

KeyboardInterrupt: 

In [9]:
tiff_data = st.two_photon.tiff_tools.read(basename_input, 
                                          metadata['size'],metadata['layout'])

In [10]:
tiff_data.shape

(2, 3280, 7, 208, 259)

In [ ]:
tiff_data = st.two_photon.tiff_tools.read(basename_input, 
                                          metadata['size'],metadata['layout'])
st.two_photon.tiff_tools.convert_to_hdf5(tiff_data,'/media/mplitt/SSD_storage/2P_scratch/test.h5')

[########################################] | 100% Completed | 152.50 s
